In [67]:
import os
import numpy as np
import random
import math
from matplotlib import pyplot as plt
import pandas as pd
from itertools import groupby
from tqdm.auto import tqdm

In [68]:
def load_pred_truth(teacher_id):
    y_pred1=np.load("y_pred_1_"+teacher_id+".npy",allow_pickle=True)
    y_true1=np.load("y_true_1_"+teacher_id+".npy",allow_pickle=True)
    y_pred2=np.load("y_pred_2_"+teacher_id+".npy",allow_pickle=True)
    y_true2=np.load("y_true_2_"+teacher_id+".npy",allow_pickle=True)
    return y_pred1,y_true1,y_pred2,y_true2

In [69]:
def group(arr):
    x=[list(j) for i, j in groupby(arr)]
    arr=[]
    s=0
    for i in range(len(x)):
        arr.append([s,s+len(x[i])-1,x[i][0]])
        s+=len(x[i])
    return arr

In [70]:
def smoothening(arr,k):
    y=[list(j) for i, j in groupby(arr)]
    arr_n=[]
    for i in range(len(y)):
        if len(y[i])<=k and i==0:
            for j in y[i]:
                arr_n.append(j)
        elif len(y[i])<=k and i==(len(y)-1):
            for j in [y[i-1][0]]:
                arr_n.append(j)
        elif len(y[i])<=k:
            for j in [y[i+1][0]]:
                arr_n.append(j)  
        else:
            for j in y[i]:
                arr_n.append(j)            
    return arr_n

In [71]:
def convert_to_timestamp(arr):
    new_arr=[]
    for i in arr:
        s=i[0]*0.01
        e=(i[1]*0.01)+0.01
        new_arr.append([np.round(s,2),np.round(e,2),i[2]])   
    return new_arr

In [72]:
def convert_to_labels(arr,label):
    new_arr=[]
    for i in arr:
        if i[-1]==0:
            new_arr.append([i[0],i[1],"N"])
        else:
            new_arr.append([i[0],i[1],label])
    return new_arr

In [73]:
def find_intersection(lst1,lst2):
    intersection=np.logical_or(lst1,lst2)
    intersection=intersection+0
    return np.array(intersection).astype(np.float32)

In [74]:
def find_tp_tn(pred,true,c):
    tp=0
    tn=0
    fp=0
    fn=0
    for i in range(len(pred)):
        if i-c<0:
            s=0
            e=i+c+1
            t=true[s:e]
        else:
            t=true[i-c:c+i+1]
        if pred[i] in t and pred[i]==1:
            tp+=1
        elif pred[i] in t and pred[i]==0:
            tn+=1
        elif pred[i] not in t and pred[i]==1:
            fp+=1
        elif  pred[i] not in t and pred[i]==0:
            fn+=1
    return tp,tn,fp,fn

In [75]:
def evaluate(pred,true,c):
    tp,tn,fp,fn=find_tp_tn(pred,true,c)
    p=tp/(tp+fp)
    r=tp/(tp+fn)
    a=(tp+tn)/(tp+tn+fp+fn+math.ulp(1.0))
    f1=2*((p*r)/(p+r))
    return p,r,a,f1

In [76]:
def get_metrics(pred,true,eta,t_collar):
    TP,TN,FP,FN = (0,0,0,0)
    event_id_pred = 0
    for event_id_true in tqdm(range(len(true))):
        if event_id_true==0:
            onset_true = float(true.iloc[event_id_true,0]) #a
            offset_true = float(true.iloc[event_id_true,1])+t_collar #b
        elif event_id_true==len(true)-1:
            onset_true = float(true.iloc[event_id_true,0])-t_collar #a
            offset_true = float(true.iloc[event_id_true,1]) #b
        else:
            onset_true = float(true.iloc[event_id_true,0])-t_collar #a
            offset_true = float(true.iloc[event_id_true,1])+t_collar #b
        gt = true.iloc[event_id_true,-1]
        delta = eta*(offset_true-onset_true) #overlap_tolerance
        onset_pred = int(onset_true*100)
        offset_pred = int(offset_true*100)
        event_occur_frequency = sum(pred[onset_pred:offset_pred])
        #print(event_occur_frequency)
        if event_occur_frequency>delta:
            if gt != 'N':
                TP+=1
            else:
                FP+=1
        else:
            if gt != 'N':
                FN+=1
            else:
                TN+=1
                
    accuracy = (TP)/(TP+FP+TN+FN)
    precision = (TP)/(TP+FP)
    recall = (TP)/(TP+FN)
    f1 = (2*precision*recall)/(precision+recall)
    print(f'TP- {TP}, FP- {FP}, FN- {FN}, FP- {FP}')
    print(f'The Accuracy is {100*accuracy}%.')
    print(f'The Precision is {100*precision}%.')
    print(f'The Recall is {100*recall}%.')
    print(f'The F1-score is {100*f1}%.')
    return accuracy, precision, recall, f1

In [77]:
def get_scores(teacher_id = '002', kappa = 1,metrics_for = 'freq',t_collar = 0, eta = 0.2):
    y_pred1,y_true1,y_pred2,y_true2 = load_pred_truth(teacher_id=teacher_id)
    if metrics_for == 'freq':
        metrics_for = 'F'
    else:
        metrics_for = 'A'
    y_pred1,y_pred2 = smoothening(y_pred1,k = kappa),smoothening(y_pred2,k = kappa)
    y_true1,y_true2=group(y_true1),group(y_true2)
    y_true1,y_true2 = convert_to_labels(y_true1,metrics_for),convert_to_labels(y_true2,metrics_for)
    y_true1 = pd.DataFrame(convert_to_timestamp(y_true1),columns = ['Onset','Offset','Mistake'])
    y_true2 = pd.DataFrame(convert_to_timestamp(y_true2),columns = ['Onset','Offset','Mistake'])
    #return y_pred1, y_true1
    if metrics_for == 'F':
        accuracy, precision, recall, f1 = get_metrics(pred = y_pred1,true = y_true1,
                                                      eta = eta, t_collar = t_collar)
    else:
        accuracy, precision, recall, f1 = get_metrics(pred = y_pred2,true = y_true2,
                                                      eta = eta, t_collar = t_collar)
    return accuracy, precision, recall, f1

In [78]:
def get_metrics_segment_wise(teacher_id,c,k):
    y_pred1,y_true1,y_pred2,y_true2=load_pred_truth(teacher_id)
    print("Results Before Smoothening- ")
    p1,r1,a1,f11=evaluate(y_pred1,y_true1,c)
    print("Frequency-")
    print("Accuracy- ",a1)
    print("Precision-",p1)
    print("Recall-",r1)
    print("F1 Score- ",f11)
    print("\n")
    p2,r2,a2,f12=evaluate(y_pred2,y_true2,c)
    print("Amplitude-")
    print("Accuracy- ",a2)
    print("Precision-",p2)
    print("Recall-",r2)
    print("F1 Score- ",f12)
    print("\n")
    y_pred1,y_pred2=smoothening(y_pred1,k),smoothening(y_pred2,k)
    print("Results After Smoothening- ")
    p1,r1,a1,f11=evaluate(y_pred1,y_true1,c)
    print("Frequency-")
    print("Accuracy- ",a1)
    print("Precision-",p1)
    print("Recall-",r1)
    print("F1 Score- ",f11)
    print("\n")
    p2,r2,a2,f12=evaluate(y_pred2,y_true2,c)
    print("Amplitude-")
    print("Accuracy- ",a2)
    print("Precision-",p2)
    print("Recall-",r2)
    print("F1 Score- ",f12)

In [79]:
teacher_id="002"

In [80]:
get_metrics_segment_wise(teacher_id,c,k)

Results Before Smoothening- 
Frequency-
Accuracy-  0.7558651026392962
Precision- 0.6696132750508116
Recall- 0.8341021016855101
F1 Score-  0.7428611141140676


Amplitude-
Accuracy-  0.8347067448680352
Precision- 0.7120606703090597
Recall- 0.31470068836270587
F1 Score-  0.43649087728067976


Results After Smoothening- 
Frequency-
Accuracy-  0.7567815249266863
Precision- 0.6702865803634745
Recall- 0.8359055281958798
F1 Score-  0.7439904309912412


Amplitude-
Accuracy-  0.8347507331378299
Precision- 0.712721627583953
Recall- 0.3143763289725015
F1 Score-  0.4363027059470815


In [89]:
print("Frequency- ")
get_scores(teacher_id, kappa = 1,metrics_for = 'freq',t_collar = 0, eta = 0.2)
print("\n")
print("Amplitude- ")
get_scores(teacher_id, kappa = 1,metrics_for = 'amp',t_collar = 0, eta = 0.2)

Frequency- 


  0%|          | 0/201 [00:00<?, ?it/s]

TP- 99, FP- 92, FN- 1, FP- 92
The Accuracy is 49.25373134328358%.
The Precision is 51.832460732984295%.
The Recall is 99.0%.
The F1-score is 68.04123711340206%.


Amplitude- 


  0%|          | 0/23 [00:00<?, ?it/s]

TP- 11, FP- 4, FN- 0, FP- 4
The Accuracy is 47.82608695652174%.
The Precision is 73.33333333333333%.
The Recall is 100.0%.
The F1-score is 84.6153846153846%.


(0.4782608695652174, 0.7333333333333333, 1.0, 0.846153846153846)